In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import scipy as sp

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster', rc={'font.size':35,
                              'axes.titlesize':50,
                              'axes.labelsize':35})

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

### potential questions

- can Meso scale predict TBI vs DC? 

- does Meso at visit one predict x outcome at visit 2 (pych and imaging outcomes) -- raw visit 2 number or 2 vs 1 difference?

- network analysis with PET data?


- role for ApoE type
- role for prazosin
- role for drinking type? MIP1a/b - VP connection?


- plasma: bFGF, VEGFA, MDC, IL1a, IL7
- CSF: CRP, MCP-1, IL1a, INFy

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/final_data/first_visits_short.csv'

In [ ]:
#read in csv containing data from all surveys and all visitseqs
data = pd.read_csv(data_path, index_col=0)
data = pd.DataFrame(data = data)

print('Original data shape:\n', data.shape, '\n')
#create class variable
data['class'] = [0 if x == 'C' else 1 for x in data['Group']]
print(data.info())
data.head(1)

In [ ]:
#explore meso data differences between groups

meso_columns = ['Plasma1_bFGF',
       'Plasma1_CRP', 'Plasma1_Eotaxin', 'Plasma1_Eotaxin3',
       'Plasma1_Flt1', 'Plasma1_ICAM1', 'Plasma1_IFNγ', 'Plasma1_IL10',
       'Plasma1_IL12_IL23p40', 'Plasma1_IL12p70', 'Plasma1_IL15',
       'Plasma1_IL16', 'Plasma1_IL17A', 'Plasma1_IL1α', 'Plasma1_IL6',
       'Plasma1_IL7', 'Plasma1_IL8', 'Plasma1_IP10', 'Plasma1_MCP1',
       'Plasma1_MCP4', 'Plasma1_MDC', 'Plasma1_MIP1α', 'Plasma1_MIP1β',
       'Plasma1_PlGF', 'Plasma1_SAA', 'Plasma1_TARC', 'Plasma1_Tie2',
       'Plasma1_TNFα', 'Plasma1_TNFβ', 'Plasma1_VCAM1', 'Plasma1_VEGF',
       'Plasma1_VEGFC', 'Plasma1_VEGFD']

for param in meso_columns:
    print(param)
    
    try:
        g = sns.catplot(x='Group', y=param, kind='bar', data=data, ci=68, height=5, aspect=4)
        plt.show()
        
        print('\n')
        
    except:
        pass

In [ ]:
#determined outliers for auditc and QBlstExp (outlier = >3 SD from mean) and remove
data = data[data["TBIID"] != 'C010']
data = data[data["TBIID"] != 'T080']

In [ ]:
#use only TBI participants
data_TBI =  data[data['Group'] == 'T']
print(data_TBI.shape)

#select second visit only
data_TBI_v2 = data_TBI[data_TBI['VisitSeq'] == 2]
print(data_TBI_v2.shape)

#get TBIIDs of participants with a second visit and use to filter at end of cell
TBIID_v2 = data_TBI_v2['TBIID'].unique()
print(len(TBIID_v2))

#combine second visit params with MESO
data_TBI_v2 = data_TBI_v2[['TBIID', 'servconn', 'cestotal',
       'BMI', 'BPSYS', 'BPDIAS', 'HRATE', 'BGlucose', 'BNa', 'BUN', 'BCreat', 'BOsmo',
       'UOsmo', 'USG', 'UNa', 'BK', 'TotalChol', 'LDL', 'HDL', 'Trig', 'CSFPROTEIN_x',
       'CSFGLUCOSE', 'CSFRBCS',  
       'CAPSTotal', 'PSQIc1', 'PSQIc2', 'PSQIc3', 'PSQIc4', 'PSQIc5',
       'PSQIc6', 'PSQIc7', 'PSQItot', 'PCLTot', 'PCL_reexp', 'PCL_avoid',
       'PCL_numb', 'PCL_hyper', 'PHQTot', 'PHQ_psych', 'PHQ_somatic',
       'NSITot', 'TBITot', 'NSI_vestibular', 'NSI_somatosensory',
       'NSI_cognitive', 'NSI_affective', 'NSI_ERP_affective',
       'NIS_ERP_vestsom', 
            'Amygdala_l', 'Amygdala_r',
       'Pallidum_l', 'Pallidum_r', 'Midbrain', 'auditc']]
print(data_TBI_v2.shape)

data_TBI_MESO = data_TBI[data_TBI['VisitSeq'] == 1][['TBIID', 'Plasma1_bFGF',
       'Plasma1_CRP', 'Plasma1_Eotaxin', 'Plasma1_Eotaxin3',
       'Plasma1_Flt1', 'Plasma1_ICAM1', 'Plasma1_IFNγ', 'Plasma1_IL10',
       'Plasma1_IL12_IL23p40', 'Plasma1_IL12p70', 'Plasma1_IL15',
       'Plasma1_IL16', 'Plasma1_IL17A', 'Plasma1_IL1α', 'Plasma1_IL6',
       'Plasma1_IL7', 'Plasma1_IL8', 'Plasma1_IP10', 'Plasma1_MCP1',
       'Plasma1_MCP4', 'Plasma1_MDC', 'Plasma1_MIP1α', 'Plasma1_MIP1β',
       'Plasma1_PlGF', 'Plasma1_SAA', 'Plasma1_TARC', 'Plasma1_Tie2',
       'Plasma1_TNFα', 'Plasma1_TNFβ', 'Plasma1_VCAM1', 'Plasma1_VEGF',
       'Plasma1_VEGFC', 'Plasma1_VEGFD']]
print(data_TBI_MESO.shape)

#combine dfs
data_TBI_v2_MESO = pd.merge(data_TBI_v2, data_TBI_MESO, how='left', on=['TBIID'], sort=False)
print(data_TBI_v2_MESO.shape)

#drop params with many missing values
data_TBI_v2_MESO.dropna(axis=1, thresh=5, inplace=True)
data_TBI_v2_MESO.head()

In [ ]:
#explore corr with MESO analytes of interest
data_TBI_v2_MESO.corr()[['Plasma1_bFGF', 'Plasma1_VEGF', 'Plasma1_MDC','Plasma1_IL1α', 'Plasma1_IL6', 'Plasma1_IL10']]
